# JupyterLab on Slurm Cluster

Install miniconda3, jupyterlab, and other packages. See [here](https://github.com/matsuolab/ucllm_nedo_prod/blob/main/infra/README.md#environment-preparation) and [here](https://www.notion.so/matsuolab-geniac/JupyterLab-96c369e0dc994dbfbb82f5c6385ba7bf?pvs=4) for setting up the environment.

Create a batch file `jupyterlab.sh`.

```
#!/bin/bash

#SBATCH --nodelist=slurm0-a3-ghpc-20
#SBATCH --time=06:00:00
#SBATCH --gpus-per-node=0
#SBATCH --cpus-per-task=64
#SBATCH --mem=64GB
#SBATCH --job-name=jupyterlab
#SBATCH --output=%x_%j.log

source $EXP_HOME/miniconda3/etc/profile.d/conda.sh
conda activate jupyter39

jupyter-lab --no-browser --port 8888 --ip 0.0.0.0
```

Then, submit the job to the cluster.

```
sbatch jupyterlab.sh
```

Find the token in the log file.

```
tail -f jupyterlab_<job_id>.log
```

Setup the IAP tunneling and SSH port forwarding to access the JupyterLab on the cluster. See [here](https://www.notion.so/matsuolab-geniac/JupyterLab-96c369e0dc994dbfbb82f5c6385ba7bf?pvs=4#926966a09dd9474385f49d130127784b) for more details.

Then open the browser and access the URL `http://localhost:8888/?token=<token>`.


In [1]:

!ls -l /storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_id = "/storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, torch_dtype="auto", device_map="auto")
tokenizer

LlamaTokenizer(name_or_path='/storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert', vocab_size=56320, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
model

Loading checkpoint shards: 100%|██████████| 41/41 [00:09<00:00,  4.51it/s]


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(56320, 2048)
    (layers): ModuleList(
      (0-39): 40 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=2048, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=2048, out_features=4096, bias=False)
              (w2): Linear(in_features=4096, out_features=2048, bias=False)
              (w3): Linear(in_features=2048, out_features=4096, bias=False)
       

In [6]:
inputs = tokenizer("今日の天気は", return_tensors="pt").to(model.device)
inputs

{'input_ids': tensor([[    1,    32, 32132, 30152, 52207, 30154]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [11]:
model.generation_config.pad_token_id = model.generation_config.eos_token_id
outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.1)
outputs

tensor([[    1,    32, 30215, 30157, 32591, 31008, 30265, 30154,   291, 32044,
         30265, 30546, 31423, 33215,   292, 32044, 30265, 30152, 48877, 30153,
         30163, 30152, 30156,   291, 30774, 30243,   417,   290,   416,   423,
           422,   512, 30152,   336, 30174, 38940, 32311,   337,   292, 32044,
         30265, 30152, 48877, 30170, 30154,   291,   419,   422,   416, 30280,
         30152, 41025, 33648, 30156, 35280, 31962,   292, 30232]])

In [8]:
tokenizer.decode(outputs[0])

'<s>今日の天気は晴れ。朝から雨が降っています。昨日の夜、雨が降ったので、今日は朝から雨になると思っていたのですが、昼前から雨が降り出しました。そして、夕方には止んで'

In [12]:
inputs = tokenizer("日本で一番高い山は", return_tensors="pt", add_special_tokens=True).to(model.device)
print("inputs: ", inputs.input_ids)
outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.1)
print("outputs: ", outputs[0])
tokenizer.decode(outputs[0], skip_special_tokens=False)

inputs:  tensor([[    1,    32, 30215, 30157, 32591, 31008, 30265, 30154]])
outputs:  tensor([    1,    32, 30215, 30157, 32591, 31008, 30265, 30154,   291, 32044,
        30265, 30546, 31423, 33215,   292, 32044, 30265, 30152, 48877, 30153,
        30163, 30152, 30156,   291, 30774, 30243,   417,   290,   416,   423,
          422,   512, 30152,   336, 30174, 38940, 32311,   337,   292, 32044,
        30265, 30152, 48877, 30170, 30154,   291,   419,   422,   416, 30280,
        30152, 41025, 33648, 30156, 35280, 31962,   292, 30232])


'<s>日本で一番高い山は、富士山だそうです。富士山の頂上にあるのが、標高1,076mの「大汝峰」。富士山の頂上からは、360度のパノラマビューが楽しめます。また'